<a href="https://colab.research.google.com/github/toddlack/colab/blob/main/ComfyUI_SDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [stable-diffusion-art.com](https://stable-diffusion-art.com) - Check out latest tutorials.

### [Guide to use this notebook](https://stable-diffusion-art.com/comfyui-colab/) - Leave comment if you have questions.

#### <b>Become a member to support the maintenance of this notebook.</b>

[<img src="https://stable-diffusion-art.com/wp-content/uploads/2023/10/see_member_benefit_button300.png" width=200>](https://stable-diffusion-art.com/member)

This notebook launches [ComfyUI](https://github.com/comfyanonymous/ComfyUI).

## Update
- 12/07/2024: Add style model folder in google drive.
- 12/02/2024: Added LTX video model.
- 11/30/2024: Fixed diffusion models linking.
- 11/16/2024: Added Mochi video model.
- 11/09/2024: Added SD3.5 medium model.


In [ ]:
#@title <font size="6" color="orange">ComfyUI</font>
#@markdown #### Follow the URL link **`https://xxxx.loca.lt`** or **`https://xxxx.ngrok-free.app`** to launch the app. The password is above the link.
#@markdown See version <a href="https://github.com/comfyanonymous/ComfyUI/releases" target="_blank">here</a>. Leave it blank for the latest version.
output_path = 'AI_PICS' #@param {type:"string"}
version='' #@param {type: "string"}

NGROK='2pwdPQZ3cS3p3m3kpy81bbqpdgc_3JC6YZhHaZNnQ3gBmR3Sd' #@param {type: "string"}
#@markdown Use NGROK if ComfyUI page has lodading issue. Get API key [here](https://dashboard.ngrok.com/get-started/your-authtoken). You can also use secrets.

#@markdown ## Only check the models you are going to use:

#@markdown ### Video models
Mochi_FP8 = False #@param{type: "boolean"}
LTX = False #@param{type: "boolean"}

#@markdown ### Flux models
Flux1_dev = True #@param{type: "boolean"}

#@markdown ### SD3 models
SD_3_5_Medium = False #@param{type: "boolean"}


#@markdown ### SDXL models
SDXL_1 = False #@param{type: "boolean"}
JuggernautXL_v8 = False #@param{type: "boolean"}
Pony_Diffusion_XL_v6 = False #@param{type: "boolean"}

#@markdown ### v1.5 models:
v1_5_model = False #@param{type: "boolean"}
Realistic_Vision_model = True #@param{type: "boolean"}
Realistic_Vision_Inpainting_model = False #@param{type: "boolean"}
DreamShaper_model = True #@param{type: "boolean"}
DreamShaper_Inpainting_model = False #@param{type: "boolean"}
Anything_v3_model = False #@param{type: "boolean"}

#@markdown ### ControlNet models:
SD_1_5_ControlNet_models = True #@param{type: "boolean"}
SDXL_ControlNet_models = False #@param{type: "boolean"}
IP_Adapter_models = True #@param{type: "boolean"}

#@markdown ### Custom Nodes:
SAVE_CUSTOM_NODES_IN_GOOGLE_DRIVE = True #@param{type: "boolean"}
#@markdown Some custom nodes don't work when saved in google drive.
ComfyUI_Manager = True #@param{type: "boolean"}
ControlNet_aux = True #@param{type: "boolean"}

#@markdown ### Extra ComfyUI arguments
Extra_arguments = '--disable-smart-memory' #@param {type: "string"}

Clear_Log = True #@param{type: "boolean"}

########## function definitions ###########
import subprocess
import threading
import time
import socket
import urllib.request
import os
import glob

def link_files(source, dest):
  '''Create symlinks for all files in the source folder to the dest folder
  Args:
    source: Absolute path of the source folder
    dest: Absolute path of the destination folder
  '''
  if not os.path.exists(source):
    !mkdir -p {source}
  if not os.path.exists(dest):
    !mkdir -p {dest}
  model_files = glob.glob(source + '/*')
  %cd {dest}
  for f in model_files:
    print(f'Linking model {f} in {dest}')
    !ln -s {f}

def initGoogleDrive(output_path, app_local_path):
  # output_path: E.g. /content/drive/MyDrive/AI_PICS
  # app_local_path: e.g. /content/ComfyUI
  app_gdrive_path = f"{output_path}/ComfyUI"
  models_gdrive_path = f"{output_path}/models"
  %cd {app_local_path}

  if not os.path.exists(output_path):
    !mkdir -p {output_path}

  if not os.path.exists(app_gdrive_path):
    !mkdir -p {app_gdrive_path}

  # create image output folder
  if not os.path.exists(image_output_path):
    !mkdir -p {image_output_path}

  # create image input folder
  if not os.path.exists(image_input_path):
    !mkdir -p {image_input_path}

  # custom nodes in google drive
  custom_nodes_local_path = app_local_path + '/custom_nodes'
  custom_nodes_gdrive_path = output_path + '/ComfyUI/custom_nodes'

  if SAVE_CUSTOM_NODES_IN_GOOGLE_DRIVE:
    if os.path.exists(custom_nodes_local_path):
      !rm -rf {custom_nodes_local_path}
    if not os.path.exists(custom_nodes_gdrive_path):
      !mkdir {custom_nodes_gdrive_path}
    !ln -s {custom_nodes_gdrive_path}
  else:
    if os.path.exists(custom_nodes_local_path):
      # remove symlink
      !rm {custom_nodes_local_path}
      !mkdir -p {custom_nodes_local_path}




  # files
  model_path_map = {
      f'{models_gdrive_path}/embeddings': f'{app_local_path}/models/embeddings',
      f'{models_gdrive_path}/animatediff': f'{app_local_path}/models/animatediff_models',
      f'{models_gdrive_path}/clip': f'{app_local_path}/models/clip',
      f'{models_gdrive_path}/clip_vision': f'{app_local_path}/models/clip_vision',
      f'{models_gdrive_path}/ControlNet': f'{app_local_path}/models/controlnet',
      f'{models_gdrive_path}/ESRGAN': f'{app_local_path}/models/upscale_models',
      f'{models_gdrive_path}/hypernetworks': f'{app_local_path}/models/hypernetworks',
      f'{models_gdrive_path}/insightface': f'{app_local_path}/models/insightface',
      f'{models_gdrive_path}/instantid': f'{app_local_path}/models/instantid',
      f'{models_gdrive_path}/layer_model': f'{app_local_path}/models/layer_model',
      f'{models_gdrive_path}/LDSR': f'{app_local_path}/models/LDSR',
      f'{models_gdrive_path}/liveportrait': f'{app_local_path}/models/liveportrait',
      f'{models_gdrive_path}/Lora': f'{app_local_path}/models/loras',
      f'{models_gdrive_path}/reactor': f'{app_local_path}/models/reactor',
      f'{models_gdrive_path}/Stable-diffusion': f'{app_local_path}/models/checkpoints',
      f'{models_gdrive_path}/unet': f'{app_local_path}/models/unet',
      f'{models_gdrive_path}/VAE': f'{app_local_path}/models/vae',
      f'{models_gdrive_path}/VAE-approx': f'{app_local_path}/models/vae',
      f'{models_gdrive_path}/xlabs/loras': f'{app_local_path}/models/xlabs/loras',
      f'{models_gdrive_path}/xlabs/controlnets': f'{app_local_path}/models/xlabs/controlnets',
      f'{models_gdrive_path}/diffusion_models': f'{app_local_path}/models/diffusion_models',
      f'{models_gdrive_path}/style_models': f'{app_local_path}/models/style_models',
  }

  # Create symbolic links for all models
  for model_path in model_path_map:
    link_files(model_path, model_path_map[model_path])

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("Tunnel Password:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')

def clear():
    from IPython.display import clear_output; return clear_output()

def downloadModel(url, rename = None):
  if 'huggingface.co' in url:
    if rename:
      filename = rename
    else:
      filename = url.split('/')[-1]
      filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    if rename:
      if '?' in url:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}&token={Civitai_API_Key}" -o {rename}
      else:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}?token={Civitai_API_Key}" -o {rename}
    else:
      if '?' in url:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}&token={Civitai_API_Key}"
      else:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}?token={Civitai_API_Key}"

def install_custom_node(url):
  %cd /content/ComfyUI/custom_nodes
  !git clone {url}

def install_custom_nodes():
  %cd {customNodeRoot}
  if ComfyUI_Manager:
    install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager')
  if ControlNet_aux:
    install_custom_node('https://github.com/Fannovel16/comfyui_controlnet_aux')



def install_models():
  %cd {ComfyUIRoot}/models/checkpoints
  print('⏳ Downloading models ...')
  # video
  if Mochi_FP8:
    downloadModel('https://huggingface.co/Comfy-Org/mochi_preview_repackaged/resolve/main/all_in_one/mochi_preview_fp8_scaled.safetensors')

  if LTX:
    %cd {ComfyUIRoot}/models/clip
    downloadModel('https://huggingface.co/Comfy-Org/stable-diffusion-3.5-fp8/resolve/main/text_encoders/t5xxl_fp16.safetensors')
    %cd {ComfyUIRoot}/models/checkpoints
    downloadModel('https://huggingface.co/Lightricks/LTX-Video/resolve/main/ltx-video-2b-v0.9.safetensors')


  # Flux
  if Flux1_dev:
    downloadModel('https://huggingface.co/Comfy-Org/flux1-dev/resolve/main/flux1-dev-fp8.safetensors')
    downloadModel('https://huggingface.co/comfyanonymous/flux_text_encoders/blob/main/clip_l.safetensors')
    downloadModel('https://huggingface.co/comfyanonymous/flux_text_encoders/blob/main/t5xxl_fp16.safetensors')

  if SD_3_5_Medium:
    downloadModel('https://huggingface.co/Comfy-Org/stable-diffusion-3.5-fp8/resolve/main/sd3.5_medium_incl_clips_t5xxlfp8scaled.safetensors')

  # SD 1.5
  if v1_5_model:
    downloadModel('https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors')

  if Realistic_Vision_model:
    downloadModel('https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1_fp16-no-ema.safetensors')

  if Realistic_Vision_Inpainting_model:
    downloadModel('https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1-inpainting.safetensors')

  if DreamShaper_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors')

  if DreamShaper_Inpainting_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_INPAINTING.inpainting.safetensors')

  if Anything_v3_model:
    downloadModel('https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors')

  # SDXL
  if SDXL_1:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors')

  if JuggernautXL_v8:
    downloadModel('https://civitai.com/api/download/models/288982')

  if Pony_Diffusion_XL_v6:
    downloadModel('https://huggingface.co/Magamanny/Pony-Diffusion-V6-XL/resolve/main/ponyDiffusionV6XL_v6StartWithThisOne.safetensors')

def installControlNetModels():
  %cd {ComfyUIRoot}/models/controlnet
  if SD_1_5_ControlNet_models:
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
    downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
    downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
    downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')


  if SDXL_ControlNet_models:
    downloadModel('https://huggingface.co/xinsir/controlnet-openpose-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors', rename='diffusion_xl_openpose.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors')
    downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors')

  if IP_Adapter_models:
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin')
    downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors')
    downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin')
    !pip install insightface

def ngrok_thread(port):
  PASSWORD='todd'
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  ngrok.set_auth_token(NGROK)
  http_tunnel = ngrok.connect(port, auth=f"{PASSWORD}:{PASSWORD}")
  print(f'Username/password: {PASSWORD}')
  print(f'ngrok public URL: {http_tunnel.public_url}')





########### Code execution ########################
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

# Get secrets
from google.colab import userdata
try:
  NGROK = userdata.get('NGROK')
  print('Used NGROK secret.')
except:
  pass

# Define folders
output_path = '/content/drive/MyDrive/' + output_path
root = '/content/'
ComfyUIRoot = root + '/ComfyUI'
customNodeRoot = ComfyUIRoot + '/custom_nodes/'
image_output_path = output_path + '/outputs'
image_input_path = output_path + '/inputs'


# Install ComfyUI
%cd {root}
!git clone https://github.com/comfyanonymous/ComfyUI.git
if version:
  !git checkout {version}
%cd {ComfyUIRoot}
!pip install -r requirements.txt

initGoogleDrive(output_path, ComfyUIRoot)

!apt-get -y install -qq aria2

# ngrok
if NGROK:
  %pip install pyngrok
  from pyngrok import ngrok

# Install models
install_models()

# Install custom nodes
install_custom_nodes()

# localtunnel

if Clear_Log:
  clear()
if NGROK:
  # ngrok
  port = 8188
  # if not 'ngrok_listener' in locals():
  #   ngrok_listener = ngrok_connect(port)
  threading.Thread(target=ngrok_thread, daemon=True, args=(port,)).start()
else:
  # Local tunnel
  !npm install -g localtunnel
  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
%cd /content/ComfyUI
!python main.py --dont-print-server --output-directory {image_output_path} --input-directory {image_input_path} {Extra_arguments}